In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [2]:
# tkn_pairs = ['HDXUSD', 'DOTUSD', 'XETHZUSD', 'DOTETH', 'XXBTZUSD', 'DOTXBT', 'XETHXXBT', 'ASTRUSD', 'CFGUSD', 'BNCUSD', 'GLMRUSD']
# op_tkn_pairs = [('HDX','USDT'), ('DOT','USDT'), ('WETH','USDT'), ('DOT','WETH'), ('WBTC','USDT'), ('DOT','WBTC'), ('WETH','WBTC'), ('ASTR','USDT'), ('CFG','USDT'), ('BNC','USDT'), ('GLMR','USDT')]
# arb_dict = {
#     ('HDX','USDT'): 'HDXUSD',
#     ('DOT','USDT'): 'DOTUSD',
#     ('WETH','USDT'): 'XETHZUSD',
#     ('DOT','WETH'): 'DOTETH',
#     ('WBTC','USDT'): 'XXBTZUSD',
#     ('DOT','WBTC'): 'DOTXBT',
#     ('WETH','WBTC'): 'XETHXXBT',
#     ('ASTR','USDT'): 'ASTRUSD',
#     ('CFG','USDT'): 'CFGUSD',
#     ('BNC','USDT'): 'BNCUSD',
#     ('GLMR','USDT'): 'GLMRUSD'
# }

# TODO move to config file
arb_list = [
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 10), 'order_book': ('HDX','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 10), 'order_book': ('DOT','USD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 4), 'order_book': ('DOT','ETH')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 3), 'order_book': ('DOT','XBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 10), 'order_book': ('ASTR','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 10), 'order_book': ('CFG','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 10), 'order_book': ('BNC','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 10), 'order_book': ('GLMR','USD')}
]

order_books = {}
for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in order_books:
        resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1])
        y = resp.json()
        order_books[tkn_pair] = y['result'][tkn_pair[0] + tkn_pair[1]]
    

# f = open('data/hdxusd_orderbook.json', 'r')
# y = json.loads(f.read())
# f = open('data/dotusd_orderbook.json', 'r')
# z = json.loads(f.read())
# hdxusd_orderbook = y['result']['HDXUSD']
# dotusd_orderbook = z['result']['DOTUSD']

# pprint(hdxusd_orderbook)



In [3]:
def get_unique_name(ls: list[str], name: str) -> str:
    if name not in ls:
        return name
    else:
        c = 1
        while name + str(c).zfill(3) in ls:
            c += 1
        return name + str(c).zfill(3)

In [4]:
RPC="wss://hydradx-rpc.dwellir.com"

chain = HydraDX(RPC) 
chain.connect()

# Omnipool metadata
# TODO: move to JSON file
metadata = [
    {'symbol': 'HDX', 'id': 0, 'decimals': 12},
    {'symbol': 'LRNA', 'id': 1, 'decimals': 12},
    {'symbol': 'DAI', 'id': 2, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 3, 'decimals': 8},
    {'symbol': 'WETH', 'id': 4, 'decimals': 18},
    {'symbol': 'DOT', 'id': 5, 'decimals': 10},
    {'symbol': 'APE', 'id': 6, 'decimals': 18},
    {'symbol': 'USDC', 'id': 7, 'decimals': 6},
    {'symbol': 'PHA', 'id': 8, 'decimals': 12},
    {'symbol': 'ASTR', 'id': 9, 'decimals': 18},
    {'symbol': 'USDT', 'id': 10, 'decimals': 6},
    {'symbol': 'IBTC', 'id': 11, 'decimals': 8},
    {'symbol': 'ZTG', 'id': 12, 'decimals': 10},
    {'symbol': 'CFG', 'id': 13, 'decimals': 18},
    {'symbol': 'BNC', 'id': 14, 'decimals': 12},
    {'symbol': 'vDOT', 'id': 15, 'decimals': 10},
    {'symbol': 'GLMR', 'id': 16, 'decimals': 18},
    {'symbol': 'INTR', 'id': 17, 'decimals': 10},
    {'symbol': 'DAI', 'id': 18, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 19, 'decimals': 8},
    {'symbol': 'WETH', 'id': 20, 'decimals': 18},
    {'symbol': 'USDC', 'id': 21, 'decimals': 6},
    {'symbol': 'USDC', 'id': 22, 'decimals': 6},
    {'symbol': 'USDT', 'id': 23, 'decimals': 6},
    ]

#Asset: 0
#	Reserve: 0
#	LRNA: 20220634396578953
#	Shares: 40600077930826935893
#	Protocol: 3694884494247885459


# asset_list = ['USDT','DOT','HDX','WETH','WBTC','IBTC','ASTR','CFG','BNC','GLMR','ZTG']
# id_list = [i for i in range(len(metadata)) if metadata[i]['symbol'] in asset_list]
# id_list = [0,5,10,4,3,11,9,13,14,16,12]

# for i in id_list:
asset_list = []
fees = {}
tokens = {}
asset_map = {}

for i in range(len(metadata)):
    try:
        state = chain.api.omnipool.asset_state(metadata[i]['id'])
    except:
        continue
        
    fee = chain.api.fees.asset_fees(metadata[i]['id'])

    tkn = get_unique_name(asset_list, metadata[i]['symbol'])
    asset_list.append(tkn)
    asset_map[i] = tkn
    tokens[tkn] = {
        'liquidity': state.reserve / 10**metadata[i]['decimals'],
        'LRNA': state.hub_reserve / 10**12
    }
    fees[tkn] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
    
# fees = {{'asset_fee': chain.api.fees.asset_fees(i)} for i in id_list}
# for i in range(len(metadata)):
#     fee = chain.api.fees.asset_fees(i)
#     fees[metadata[i]['symbol']] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
    
chain.close()
# pprint(fees)

# get Omnipool data
# tokens = {
#     'USDT': {
#         'liquidity': 2062772,
#         'LRNA': 2062772
#     },
#     'DOT': {
#         # 'liquidity': 389000,
#         'liquidity': 350000,
#         'LRNA': 1456248
#     },
#     'HDX': {
#         'liquidity': 108000000,
#         'LRNA': 494896
#     }
# }

pprint(tokens)
pprint(asset_map)


{'ASTR': {'LRNA': 9168.686628028167, 'liquidity': 4560797.91676211},
 'BNC': {'LRNA': 7139.95301821198, 'liquidity': 702797.3111294416},
 'CFG': {'LRNA': 8824.522203223809, 'liquidity': 603921.8080597813},
 'DAI': {'LRNA': 13087.906345325118, 'liquidity': 333449.16118631966},
 'DAI001': {'LRNA': 31626.944194316842, 'liquidity': 804832.3731989582},
 'DOT': {'LRNA': 70688.0051392197, 'liquidity': 421708.0248276837},
 'GLMR': {'LRNA': 5361.714075194796, 'liquidity': 663370.1851263171},
 'HDX': {'LRNA': 19189.482084954576, 'liquidity': 119900761.79399426},
 'IBTC': {'LRNA': 35124.14259846746, 'liquidity': 25.76875101},
 'USDT': {'LRNA': 84597.21509506422, 'liquidity': 2170044.892663},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 42052.70022286926, 'liquidity': 30.90413606},
 'WETH': {'LRNA': 21609.26712492295, 'liquidity': 308.3879667597883},
 'WETH001': {'LRNA': 71522.49491638002, 'liquidity': 1018.0452401477638},
 'ZTG': {'LRNA': 4310.262791368341, 'l

In [5]:

# dot_usdt_order_book = {
#     'bids': [{'price': 3.60, 'amount': 200},
#              {'price': 3.59, 'amount': 100},
#              {'price': 3.50, 'amount': 100},
#              {'price': 3.40, 'amount': 2000}],
#     'asks': [{'price': 3.70, 'amount': 100},
#              {'price': 3.74, 'amount': 5000},
#              {'price': 3.80, 'amount': 200},
#              {'price': 3.90, 'amount': 2000}]
# }
# 
# hdx_usdt_order_book = {
#     'bids': [{'price': 0.03, 'amount': 2000},
#              {'price': 0.025, 'amount': 2000},
#              {'price': 0.02, 'amount': 2000},
#              {'price': 0.015, 'amount': 2000}],
#     'asks': [{'price': 0.04, 'amount': 2000},
#              {'price': 0.05, 'amount': 2000},
#              {'price': 0.06, 'amount': 2000},
#              {'price': 0.07, 'amount': 2000}]
# }

# hdx_dot_order_book = {
#     'bids': [{'price': 0.005, 'amount': 2000},
#              {'price': 0.004, 'amount': 2000}],
#     'asks': [{'price': 0.0052, 'amount': 2000},
#              {'price': 0.0055, 'amount': 2000}]
# }

ob_objs = {}
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs[tkn_pair] = ob_obj
    
order_book_asset_list = []
for tkn_pair in order_books:
    for tkn in tkn_pair:
        if tkn not in order_book_asset_list:
            order_book_asset_list.append(tkn)

# order_book = {}
# for i in range(len(arb_list)):
#     base_id, quote_id = arb_list[i]['tkn_ids']
#     orderbook_tkn_pair = arb_list[i]['order_book']
#     tkn_pair = (asset_map[base_id], asset_map[quote_id])
#     order_book[tkn_pair] = orderbook_tkn_pair 
#     
pprint(ob_objs)

# order_book = {op_tkn_pairs[i]: ob_objs[i] for i in range(len(op_tkn_pairs))}

# order_book = {
#     ('DOT','USDT'): dot_usdt_order_book_obj,
#     ('HDX','USDT'): hdx_usdt_order_book_obj,
#     # ('HDX','DOT'): hdx_dot_order_book
# }



lrna_fee = 0.0005
asset_fee = 0.0025
cex_fee = 0.0016
buffer = 0.0002

# lrna_fee = 0.0000
# asset_fee = 0.0000
# cex_fee = 0.0000
# buffer = 0.0

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


{('ASTR', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe14d04db20>,
 ('BNC', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe14d062fa0>,
 ('CFG', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe14d04d2e0>,
 ('DOT', 'ETH'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe1647f36a0>,
 ('DOT', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe1647f3880>,
 ('DOT', 'XBT'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe14d04d910>,
 ('GLMR', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe1742ce730>,
 ('HDX', 'USD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe17433abb0>,
 ('XETH', 'XXBT'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe14d04d880>,
 ('XETH', 'ZUSD'): <hydradx.model.amm.centralized_market.OrderBook object at 0x7fe1647f3730>,
 ('XXBT', 'ZUSD'): <hydradx.model.amm.centralized_market.OrderBook object 

In [6]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    tkn_pair = (asset_map[base_id], asset_map[quote_id])
    order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('WBTC', 'USDT'): ('XXBT', 'ZUSD'),
 ('WETH', 'USDT'): ('XETH', 'ZUSD'),
 ('WETH', 'WBTC'): ('XETH', 'XXBT')}


In [7]:
all_swaps = get_arb_swaps(op_state, cex, order_book_map, buffer=buffer)
for tkn_pair in all_swaps:
    if all_swaps[tkn_pair] != []:
        print(tkn_pair)
        print(all_swaps[tkn_pair])

('WETH', 'USDT')
[{'dex': {'trade': 'sell', 'buy_asset': 'USDT', 'sell_asset': 'WETH', 'price': 1788.05, 'amount': 0.05355854034423828, 'min_buy': 95.94925318754912}, 'cex': {'trade': 'buy', 'buy_asset': 'XETH', 'sell_asset': 'ZUSD', 'price': 1788.228805, 'amount': 0.05355854034423828}}]
('WBTC', 'USDT')
[{'dex': {'trade': 'buy', 'buy_asset': 'WBTC', 'sell_asset': 'USDT', 'price': 34710.3, 'amount': 0.001, 'max_sell': -65.51124195073842}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XXBT', 'price': 34706.82897, 'amount': 0.001}}, {'dex': {'trade': 'buy', 'buy_asset': 'WBTC', 'sell_asset': 'USDT', 'price': 34708.6, 'amount': 0.002199077033615113, 'max_sell': 6.763721615505219}, 'cex': {'trade': 'sell', 'buy_asset': 'ZUSD', 'sell_asset': 'XXBT', 'price': 34705.12914, 'amount': 0.002199077033615113}}]
('DOT', 'WBTC')
[{'dex': {'trade': 'sell', 'buy_asset': 'WBTC', 'sell_asset': 'DOT', 'price': 0.00012361, 'amount': 0.3606605529785156, 'min_buy': 0.003243479597568512}, 'cex

In [8]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
execute_arb(state, cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'WETH001': 'ETH',
             'IBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            
# for tkn_pair in all_swaps:
#     print(tkn_pair)
#     op_spot = OmnipoolState.price(state, tkn_pair[0], tkn_pair[1])
#     buy_spot = op_spot / ((1 - lrna_fee)*(1 - asset_fee))
#     sell_spot = op_spot * (1 - lrna_fee) * (1 - asset_fee)
#     print((sell_spot, op_spot, buy_spot))
    


{'HDX': 0, 'USD': 8.21858024597168, 'BTC': 1.1920928955078125e-07, 'ETH': 0.0, 'DOT': 0.0, 'ASTR': 0, 'ZTG': 0, 'CFG': 0, 'BNC': 0.0, 'vDOT': 0, 'GLMR': 0}
